In [1]:
import pickle
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from importlib.metadata import version

print("Pandas version: " + str(pd.__version__))
print("Numpy version: " + str(np.__version__))
print("Scikit-learn version: " + str(version("scikit-learn")))

Pandas version: 2.3.3
Numpy version: 2.3.4
Scikit-learn version: 1.7.2


In [2]:
# load preprocessed test data
with open('../data/temp/target_encoded_data.pkl', 'rb') as f:
    data = pickle.load(f)

X_test_encoded = data['X_test']
X_test_orig = data['X_test_orig']

y_test = data['y_test']

In [3]:
# load ols model
with open('../models/ols_model.pkl', 'rb') as f:
    ols_saved = pickle.load(f)

# evaluate OLS model on test data
y_test_orig = np.exp(y_test)

# get features used by OLS
ols_features = ols_saved['feature_names']
X_test_ols = X_test_encoded[ols_features]

# standardize
X_test_ols_scaled = ols_saved['scaler'].transform(X_test_ols)

# Convert back to DataFrame to preserve feature names
X_test_ols_scaled = pd.DataFrame(
    X_test_ols_scaled, 
    columns = ols_features,
    index = X_test_ols.index
)

# predict (on log scale)
y_test_pred_ols_log = ols_saved['ols_model'].predict(X_test_ols_scaled)

# transform to original scale
y_test_pred_ols = np.exp(y_test_pred_ols_log)

# evaluate
mae_ols = mean_absolute_error(y_test_orig, y_test_pred_ols)
rmse_ols = np.sqrt(mean_squared_error(y_test_orig, y_test_pred_ols))
r2_ols = r2_score(y_test_orig, y_test_pred_ols)

print(f"MAE:  {mae_ols:.3f} days")
print(f"RMSE: {rmse_ols:.3f} days")
print(f"R2:   {r2_ols:.3f}")

MAE:  2.300 days
RMSE: 5.771 days
R2:   0.125


In [4]:
with open('../models/rf_model.pkl', 'rb') as f:
    rf_saved = pickle.load(f)

# predict (on log scale)
y_test_pred_rf = rf_saved['rf_model'].predict(X_test_encoded)

# transform to original scale
# y_test_pred_rf = np.exp(y_test_pred_rf_log)

# evaluate
mae_rf = mean_absolute_error(y_test_orig, y_test_pred_rf)
rmse_rf = np.sqrt(mean_squared_error(y_test_orig, y_test_pred_rf))
r2_rf = r2_score(y_test_orig, y_test_pred_rf)

print(f"MAE:  {mae_rf:.3f} days")
print(f"RMSE: {rmse_rf:.3f} days")
print(f"R2:   {r2_rf:.3f}")

MAE:  2.397 days
RMSE: 5.843 days
R2:   0.103


In [5]:
with open('../models/catboost_model.pkl', 'rb') as f:
    catboost_saved = pickle.load(f)

# predict (already on original scale)
y_test_pred_cat = catboost_saved['catboost_model'].predict(X_test_orig)

# evaluate
mae_cat = mean_absolute_error(y_test_orig, y_test_pred_cat)
rmse_cat = np.sqrt(mean_squared_error(y_test_orig, y_test_pred_cat))
r2_cat = r2_score(y_test_orig, y_test_pred_cat)

print(f"MAE:  {mae_cat:.3f} days")
print(f"RMSE: {rmse_cat:.3f} days")
print(f"R2:   {r2_cat:.3f}")

MAE:  2.371 days
RMSE: 5.839 days
R2:   0.105


In [ ]:
# create the model pipeline

In [ ]:
# investigate why test r2 is so low
X_test = data['X_test']
y_test = data['y_test']
y_test_orig = np.exp(y_test)

# Select features
X_test_reduced = X_test[retained_features]

# Use TRANSFORM not FIT_TRANSFORM
X_test_scaled = scaler.transform(X_test_reduced)  # ← FIXED
X_test_scaled = pd.DataFrame(X_test_scaled, columns=retained_features, index=X_test_reduced.index)

# Predict (log scale)
y_test_pred_log_std = ols_standardized.predict(X_test_scaled)

# Transform to original scale
y_test_pred_std = np.exp(y_test_pred_log_std)

# Evaluate with y_test_orig, not y_val_orig!
rmse_std = np.sqrt(mean_squared_error(y_test_orig, y_test_pred_std))  # ← FIXED
mae_std = mean_absolute_error(y_test_orig, y_test_pred_std)  # ← FIXED
r2_std = r2_score(y_test_orig, y_test_pred_std)  # ← FIXED

print("\nOLS with standardized features (evaluated on original scale):")
print(f"RMSE: {rmse_std:.3f}, MAE: {mae_std:.3f}, R²: {r2_std:.3f}")

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Compare distributions
print("="*80)
print("TARGET VARIABLE COMPARISON")
print("="*80)

# Get original scale targets
y_train_orig = np.exp(y_train)
y_val_orig = np.exp(y_val)
y_test_orig = np.exp(y_test)

# Basic statistics
print("\nBasic Statistics:")
print(f"{'Metric':<20} {'Train':<15} {'Validation':<15} {'Test':<15}")
print("-"*70)
print(f"{'Mean':<20} {y_train_orig.mean():<15.2f} {y_val_orig.mean():<15.2f} {y_test_orig.mean():<15.2f}")
print(f"{'Std Dev':<20} {y_train_orig.std():<15.2f} {y_val_orig.std():<15.2f} {y_test_orig.std():<15.2f}")
print(f"{'Variance':<20} {y_train_orig.var():<15.2f} {y_val_orig.var():<15.2f} {y_test_orig.var():<15.2f}")
print(f"{'Min':<20} {y_train_orig.min():<15.2f} {y_val_orig.min():<15.2f} {y_test_orig.min():<15.2f}")
print(f"{'Max':<20} {y_train_orig.max():<15.2f} {y_val_orig.max():<15.2f} {y_test_orig.max():<15.2f}")
print(f"{'Median':<20} {y_train_orig.median():<15.2f} {y_val_orig.median():<15.2f} {y_test_orig.median():<15.2f}")
print(f"{'Q1':<20} {y_train_orig.quantile(0.25):<15.2f} {y_val_orig.quantile(0.25):<15.2f} {y_test_orig.quantile(0.25):<15.2f}")
print(f"{'Q3':<20} {y_train_orig.quantile(0.75):<15.2f} {y_val_orig.quantile(0.75):<15.2f} {y_test_orig.quantile(0.75):<15.2f}")

# Key insight: Check if test has more variance
print("\n" + "="*80)
print("VARIANCE RATIO (higher = more variable)")
print("="*80)
print(f"Test variance / Val variance: {y_test_orig.var() / y_val_orig.var():.2f}x")
print(f"Test variance / Train variance: {y_test_orig.var() / y_train_orig.var():.2f}x")

In [ ]:
# Plot distributions
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Histograms
axes[0, 0].hist(y_train_orig, bins=50, alpha=0.5, label='Train', edgecolor='black')
axes[0, 0].hist(y_val_orig, bins=50, alpha=0.5, label='Validation', edgecolor='black')
axes[0, 0].hist(y_test_orig, bins=50, alpha=0.5, label='Test', edgecolor='black')
axes[0, 0].set_xlabel('Length of Stay (days)')
axes[0, 0].set_ylabel('Frequency')
axes[0, 0].set_title('Distribution Comparison')
axes[0, 0].legend()

# Box plots
axes[0, 1].boxplot([y_train_orig, y_val_orig, y_test_orig], 
                    labels=['Train', 'Validation', 'Test'])
axes[0, 1].set_ylabel('Length of Stay (days)')
axes[0, 1].set_title('Box Plot Comparison')

# Log scale histograms
axes[1, 0].hist(y_train, bins=50, alpha=0.5, label='Train', edgecolor='black')
axes[1, 0].hist(y_val, bins=50, alpha=0.5, label='Validation', edgecolor='black')
axes[1, 0].hist(y_test, bins=50, alpha=0.5, label='Test', edgecolor='black')
axes[1, 0].set_xlabel('Log(Length of Stay)')
axes[1, 0].set_ylabel('Frequency')
axes[1, 0].set_title('Distribution Comparison (Log Scale)')
axes[1, 0].legend()

# Cumulative distributions
axes[1, 1].hist(y_train_orig, bins=100, cumulative=True, density=True, 
                histtype='step', linewidth=2, label='Train')
axes[1, 1].hist(y_val_orig, bins=100, cumulative=True, density=True, 
                histtype='step', linewidth=2, label='Validation')
axes[1, 1].hist(y_test_orig, bins=100, cumulative=True, density=True, 
                histtype='step', linewidth=2, label='Test')
axes[1, 1].set_xlabel('Length of Stay (days)')
axes[1, 1].set_ylabel('Cumulative Probability')
axes[1, 1].set_title('Cumulative Distribution')
axes[1, 1].legend()

plt.tight_layout()
plt.show()

In [ ]:
# Count extreme values
print("\n" + "="*80)
print("EXTREME VALUES")
print("="*80)

thresholds = [10, 20, 30, 50, 100]
for threshold in thresholds:
    train_count = (y_train_orig > threshold).sum()
    val_count = (y_val_orig > threshold).sum()
    test_count = (y_test_orig > threshold).sum()
    
    train_pct = train_count / len(y_train_orig) * 100
    val_pct = val_count / len(y_val_orig) * 100
    test_pct = test_count / len(y_test_orig) * 100
    
    print(f"\nLoS > {threshold} days:")
    print(f"  Train: {train_count} ({train_pct:.1f}%)")
    print(f"  Val:   {val_count} ({val_pct:.1f}%)")
    print(f"  Test:  {test_count} ({test_pct:.1f}%)")

In [ ]:
# Understand R² breakdown
print("\n" + "="*80)
print("R² BREAKDOWN")
print("="*80)

def r2_components(y_true, y_pred):
    ss_res = ((y_true - y_pred) ** 2).sum()  # Residual sum of squares
    ss_tot = ((y_true - y_true.mean()) ** 2).sum()  # Total sum of squares
    r2 = 1 - (ss_res / ss_tot)
    return ss_res, ss_tot, r2

# Validation
y_val_pred = np.exp(ols_standardized.predict(X_val_scaled))
ss_res_val, ss_tot_val, r2_val = r2_components(y_val_orig, y_val_pred)

# Test
y_test_pred = np.exp(y_test_pred_log_std)
ss_res_test, ss_tot_test, r2_test = r2_components(y_test_orig, y_test_pred)

print(f"{'Metric':<25} {'Validation':<20} {'Test':<20}")
print("-"*65)
print(f"{'SS_residual':<25} {ss_res_val:<20.2f} {ss_res_test:<20.2f}")
print(f"{'SS_total':<25} {ss_tot_val:<20.2f} {ss_tot_test:<20.2f}")
print(f"{'R²':<25} {r2_val:<20.3f} {r2_test:<20.3f}")
print(f"\nSS_total ratio (test/val): {ss_tot_test / ss_tot_val:.2f}x")
print(f"SS_residual ratio (test/val): {ss_res_test / ss_res_val:.2f}x")

print("\nInterpretation:")
if ss_tot_test / ss_tot_val > 1.5:
    print("  ⚠️  Test set has MUCH higher total variance")
    print("     This means test data is more spread out, making it harder to explain")

In [ ]:
# Compare prediction quality
print("\n" + "="*80)
print("PREDICTION QUALITY")
print("="*80)

# Correlation between predictions and actuals
from scipy.stats import pearsonr

corr_val, _ = pearsonr(y_val_orig, y_val_pred)
corr_test, _ = pearsonr(y_test_orig, y_test_pred)

print(f"Correlation (predictions vs actuals):")
print(f"  Validation: {corr_val:.3f}")
print(f"  Test: {corr_test:.3f}")

# Scatter plots
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].scatter(y_val_orig, y_val_pred, alpha=0.3)
axes[0].plot([0, y_val_orig.max()], [0, y_val_orig.max()], 'r--', label='Perfect prediction')
axes[0].set_xlabel('Actual LoS (days)')
axes[0].set_ylabel('Predicted LoS (days)')
axes[0].set_title(f'Validation Set (R²={r2_val:.3f})')
axes[0].legend()

axes[1].scatter(y_test_orig, y_test_pred, alpha=0.3)
axes[1].plot([0, y_test_orig.max()], [0, y_test_orig.max()], 'r--', label='Perfect prediction')
axes[1].set_xlabel('Actual LoS (days)')
axes[1].set_ylabel('Predicted LoS (days)')
axes[1].set_title(f'Test Set (R²={r2_test:.3f})')
axes[1].legend()

plt.tight_layout()
plt.show()

In [ ]:
X_val_orig = data['X_val_orig']
X_test_orig = data['X_test_orig']

# Compare key features between validation and test
print("\n" + "="*80)
print("FEATURE DISTRIBUTION: VALIDATION vs TEST")
print("="*80)

key_features = retained_features

for feature in key_features:
    print(f"\n{feature}:")
    
    val_dist = X_val_orig[feature].value_counts(normalize=True).sort_index()
    test_dist = X_test_orig[feature].value_counts(normalize=True).sort_index()
    
    comparison = pd.DataFrame({
        'Validation %': val_dist * 100,
        'Test %': test_dist * 100,
        'Difference': (test_dist - val_dist) * 100
    }).fillna(0)
    
    print(comparison.round(1))
    
    # Statistical test
    from scipy.stats import chi2_contingency

    # Build contingency table
    val_counts = X_val_orig[feature].value_counts().reindex(common, fill_value=0)
    test_counts = X_test_orig[feature].value_counts().reindex(common, fill_value=0)

    # Remove categories where BOTH sets have 0
    mask = ~((val_counts == 0) & (test_counts == 0))
    val_counts = val_counts[mask]
    test_counts = test_counts[mask]

    # If fewer than 2 categories remain, skip test
    if len(val_counts) < 2:
        print("  ⚠️ Not enough non-zero categories for chi-square test")
        continue

    contingency = np.vstack([val_counts.values, test_counts.values])

    chi2, p, dof, expected = chi2_contingency(contingency)

    if p < 0.05:
        print(f"  ⚠️  SIGNIFICANTLY DIFFERENT (p={p:.4f})")
    else:
        print(f"  ✓ Similar distributions (p={p:.4f})")